In [1]:
import pandas as pd
import numpy as np

In [ ]:
# C:/Workspace/02.DataAnalysis/07.NLP/word2vec-nlp-tutorial/

In [2]:
df = pd. read_csv('C:/Workspace/02.DataAnalysis/07.NLP/word2vec-nlp-tutorial/labeledTrainData.tsv/labeledTrainData.tsv',sep='\t',quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [5]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## 1. 전처리

In [3]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />',' ')

In [4]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]', ' ',x))

In [7]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## 2. 학습 및 테스트 데이터 셋으로 분리

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment,test_size=0.2,                   # 20000-train, 5000-test
    stratify=df.sentiment, random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

## 3. 텍스트 변환

- CountVectorizer, ngram_range(1,1)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 이렇게 하면 안됨
cvect1 = CountVectorizer(stop_words='english')
X-train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)

In [11]:
# train, test data set에 있는 동일한 단어에 대해서 동일한 값으로 변환
cvect1 = CountVectorizer(stop_words='english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)


In [13]:
X_test_cv1.shape,X_train_cv1.shape

((5000, 66971), (20000, 66971))

In [23]:
cvect1 = CountVectorizer(stop_words='english',ngram_range=(1,2))
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)

## 로지스틱 회귀

In [14]:
from sklearn.linear_model import LogisticRegression

In [16]:
# 훈련(X,y 모두 필요)
lr1 = LogisticRegression(max_iter=500)
lr1.fit(X_train_cv1, y_train)

LogisticRegression(max_iter=500)

In [24]:
lr2 = LogisticRegression(max_iter=500)
lr2.fit(X_train_cv1, y_train)

LogisticRegression(max_iter=500)

In [17]:
# 테스트 데이터에 대해 예측
pred1 = lr1.predict(X_test_cv1)

In [25]:
pred2 = lr2.predict(X_test_cv1)

## 평가

In [18]:
from sklearn.metrics import accuracy_score

In [26]:
score1 = accuracy_score(y_test, pred1)
score1

0.8832